In [45]:
import pandas as pd
from pdfquery import PDFQuery
import re

In [46]:
pdf = PDFQuery("resultado1570777439.pdf")
pdf.load()

In [44]:
textos = pdf.tree.xpath("//text()")
textos = [t for t in textos if re.match("\d", t)]

['22/12/2001 ',
 '1570777439 ',
 '24/03/2024 ',
 '208 mg/dL Maior ou igual a 20 anos: ',
 '60 mg/dL Maior ou igual a 20 anos: ',
 '22/12/2001 ',
 '1570777439 ',
 '24/03/2024 ',
 '72 mg/dL Maior ou igual a 20 anos: ',
 '15 mg/dL Maior ou igual a 20 anos: ',
 '22/12/2001 ',
 '1570777439 ',
 '24/03/2024 ',
 '121 mg/dL Maior ou igual a 20 anos: ',
 '22/12/2001 ',
 '1570777439 ',
 '24/03/2024 ',
 '136 mg/dL Maior ou igual a 20 anos: ',
 '83 mg/dL 70 a 99 mg/dL ',
 '22/12/2001 ',
 '1570777439 ',
 '24/03/2024 ',
 '5,7 % ',
 '5,7 a 6,4% - risco aumentado para diabetes ',
 '22/12/2001 ',
 '1570777439 ',
 '24/03/2024 ',
 '25 HIDROXI-VITAMINA D, soro ',
 '23 ng/mL População saudável (até 60 anos): acima de 20 ng/mL* ',
 '2020;64/4 :462. ',
 '336 ng/L Normal : maior que 300 ng/L ',
 '22/12/2001 ',
 '1570777439 ',
 '24/03/2024 ',
 '0,69 mUI/L 20 - 59 a: 0,45 a 4,5 mUI/L ',
 '1,2 ng/dL Acima de 20 a: 0,9 a 1,7 ng/dL ',
 '1o. trimestre: 0,9 a 1,5 ng/dL3o. trimestre: 0,6 a 1,2 ng/dL ',
 '22/12/2001 ',